In [1]:
import pandas as pd

In [2]:
df_predictions = pd.read_csv("df_predictions.csv")

In [3]:
df = pd.read_csv("kidsInMindSubtitles2004.csv")
df.shape

(2004, 15)

In [10]:
query = "Given the subtitle rating of a movie = "+str(round(df_predictions['Predcited_Score'][0], 1))+" \
; and a list of inappropriate words : "+ df_predictions['inappropriate_words'][0] + " Generate \
language summary like kids in mind website? Please consider the below two example language summaries of \
movies:  \n 1." + df['Language_Description'][1] + "\n 2."+df['Language_Description'][1]
query

"Given the subtitle rating of a movie = 6.5 ; and a list of inappropriate words : ['bondage', 'bitch', 'ass', 'fuck', 'fuck', 'goddamn', 'fat', 'shit', 'masturbate', 'hell', 'fuck', 'fat', 'fat', 'toilet', 'dead', 'skank', 'prick', 'ass', 'fuck', 'cock', 'toilet', 'toilet', 'toilet', 'fucking', 'tinkle', 'ugly', 'butt', 'stupid', 'stupid', 'hell', 'fucked', 'killing', 'fuck', 'sex', 'tits', 'bulldyke', 'damn', 'fat', 'stupid', 'ass', 'fucking', 'piss', 'desire', 'desire', 'desire', 'burn', 'tongue'] Generate language summary like kids in mind website? Please consider the below two example language summaries of Please consider the below two example language summaries of movies:  \n 1. About 67 F-words and its derivatives, 2 obscene hand gestures, 12 sexual references, 38 scatological terms, 38 anatomical terms, 39 mild obscenities, name-calling (crazy, moron, idiots, dumb, sluts, stupid, psycho, punk, Negro, drunks, school boy, boring, pussy whipped, mini-mite, corny, Little Miss Overac

In [4]:


import os
import openai

os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"


openai.organization = ""

openai.api_key = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"



#openai.Model.list()



In [5]:


response_id_done = {}
import re
import json

import openai  # for OpenAI API calls
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff


@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)

In [6]:

from openai import OpenAI
os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"


client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    # api_key="My API Key",
)
from docx import Document

def transcribe_audio(audio_file_path):
    with open(audio_file_path, 'rb') as audio_file:
        transcription = client.audio.transcriptions.create("whisper-1", audio_file)
    return transcription['text']

In [8]:
model_name = 'gpt-3.5'

In [12]:
response = client.chat.completions.create( model="gpt-3.5-turbo",
                                          temperature=0,
                                          messages=[{
                                              "role": "system",
                                              "content": query, 
                                          }
                                          ]
                                         )

In [16]:
print(response.dict()['choices'][0]['message']['content'])

The movie has a subtitle rating of 6.5. It contains several inappropriate words and phrases, including "bondage," "bitch," "ass," "fuck," "goddamn," "fat," "shit," "masturbate," "hell," "toilet," "dead," "skank," "prick," "cock," "fucking," "tinkle," "ugly," "butt," "stupid," "fucked," "killing," "sex," "tits," "bulldyke," "damn," "piss," "desire," and "burn." These words and phrases may not be suitable for younger audiences.


/tmp/ipykernel_535872/2686690461.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  print(response.dict()['choices'][0]['message']['content'])


In [17]:
df_predictions['gpt_3.5_response'] = None

In [19]:
from tqdm import tqdm

In [20]:

for idx in tqdm(range(len(df_predictions))):
    
    #idx += 24
    
    query = "Given the subtitle rating of a movie = "+str(round(df_predictions['Predcited_Score'][idx], 1))+" \
    ; and a list of inappropriate words : "+ df_predictions['inappropriate_words'][idx] + " Generate \
    language summary like kids in mind website? Please consider the below two example language summaries of \
    movies:  \n 1." + df['Language_Description'][0] + "\n 2."+df['Language_Description'][1]
    
    response = client.chat.completions.create( model="gpt-3.5-turbo",
                                          temperature=0,
                                          messages=[{
                                              "role": "system",
                                              "content": query, 
                                          }
                                          ]
                                         )
    
    with open("gpt_3.5_response.json", "a") as file:
        file.write(json.dumps(str(idx)+":"+str(response)+"\n"))
        
    df_predictions['gpt_3.5_response'][idx] = response.dict()['choices'][0]['message']['content']
    

  0%|                                                                                                                                  | 0/201 [00:00<?, ?it/s]/tmp/ipykernel_535872/142966323.py:23: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  df_predictions['gpt_3.5_response'][idx] = response.dict()['choices'][0]['message']['content']
/tmp/ipykernel_535872/142966323.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictions['gpt_3.5_response'][idx] = response.dict()['choices'][0]['message']['content']
  0%|▌                                                                                                                       

In [32]:
df_predictions.to_csv("df_predictions_gpt_3.5_response.csv", index = False)

In [22]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [23]:
scores = scorer.score(df_predictions['language_summary'][0], df_predictions['gpt_3.5_response'][0])

In [24]:
scores

{'rouge1': Score(precision=0.2542372881355932, recall=0.2631578947368421, fmeasure=0.25862068965517243),
 'rouge2': Score(precision=0.034482758620689655, recall=0.03571428571428571, fmeasure=0.03508771929824562),
 'rougeL': Score(precision=0.11864406779661017, recall=0.12280701754385964, fmeasure=0.12068965517241378)}

In [25]:

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for idx in range(len(df_predictions)):
    
    scores = scorer.score(df_predictions['language_summary'][idx], df_predictions['gpt_3.5_response'][idx])
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)


In [26]:
sum(rougeL_scores)/len(rougeL_scores)

0.10716993284465803

In [27]:
#0.1071

In [28]:
references = list(df_predictions['language_summary'])
summaries = list(df_predictions['gpt_3.5_response'])

In [30]:
import bert_score

In [31]:
P, R, F1 = bert_score.score(summaries, references, lang='en')
bertscore_precision = P.mean().item()
bertscore_recall = R.mean().item()
bertscore_f1 = F1.mean().item()

print("BERTScore Precision:", bertscore_precision)
print("BERTScore Recall:", bertscore_recall)
print("BERTScore F1:", bertscore_f1)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore Precision: 0.824908971786499
BERTScore Recall: 0.8517156839370728
BERTScore F1: 0.8376520276069641
